In [ ]:
from hello.hello3 import HelloApp
from hello import hello3
from hello import hello
from hello.pid.ag_pid import agpid
from hello.pid.ag_pid.agpid import *

In [ ]:
def runagpid(r):
    try:
        r.doall()
    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        return r

    
def run_80Ltests(ip='71.189.82.196:85'):
    p = 60
    i = 19
    d = 0
    app = hello3.open_hello(ip)
    t1 = TempAutoToAutoTest(p, i, d, 39, PIDTest.CURRENT_PV, app, name="80L Temp Verif")
    t2 = TempAutoToAutoTest(p, i, d, 37, 39, app, name="80L Temp Verif")   
    r = PIDRunner([t1, t2], wb_name='temp_80L_test.xlsx')
    return runagpid(r)
    

In [3]:
#r = run_80Ltests()
_minute = 60
_hour = 60* _minute

In [4]:
def a2a_fmt(a2a):
    # reference for A2A configuration Settings
    a2a.pretest_stabilize           = 30 * _minute
    a2a.pretest_stabilize_timeout   = 60 * _minute
    a2a.pretest_stabilize_margin    = 0.2
    a2a.settle_time                 = 30 * _minute
    a2a.timeout                     = 3 * _hour
    a2a.error_band                  = 0.1
    a2a.mintime                     = 2 * _hour
    a2a.agitation_sp                = 12


def run_80Ltests2(ip='71.189.82.196:85'):
    TempOffToAutoTest.SHORT_NAME = "O2A"  # hotfix
    p = 60
    i = 35
    d = 0
    app = hello3.open_hello(ip)
    app.retry_count = -1
    
    t0 = TempOffToAutoTest(p, i, d, 26, 37, app)
    t0.pretest_stabilize_timeout   = 18 * _hour
    t0.pretest_stabilize_margin    = 0.2
    t0.settle_time                 = 60 * _minute
    t0.timeout                     = 3 * _hour
    t0.error_band                  = 0.1
    t0.mintime                     = 3 * _hour
    t0.wait_for_cooling            = True
    t0.agitation_sp                = 12
    
    # t0 sets up t1, t1 sets up t2
    # so pretest stable time can be short
    # previous testing showed that mintime
    # was a bit short, so bump to 2 hr and 
    # shrink error band to 0.1
    t1 = TempAutoToAutoTest(p, i, d, 37, 35, app)
    t1.pretest_stabilize = 10*_minute
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    
    t2 = TempAutoToAutoTest(p, i, d, 35, 37, app)   
    t2.pretest_stabilize = 10 * _minute
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    
    r = PIDRunner([t0, t1, t2], wb_name='temp_80L_test_80L_a2a_60.35.0.xlsx')
    return runagpid(r)


In [5]:
def run_80Ltests3():
    ip='71.189.82.196:85'
    p = 60
    i = 35
    d = 0
    app = hello3.open_hello(ip)
    app.retry_count = -1
    r2 = PIDRunner(wb_name='temp_80L_test_80L_a2a_60.35.0 worst case.xlsx')
    t1 = TempAutoToAutoTest(p, i, d, 37, 35.8, app)
    t1.pretest_stabilize = 10*_minute
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    t1.settle_time = _hour * 1
    
    t2 = TempAutoToAutoTest(p, i, d, 35.8, 37, app)   
    t2.pretest_stabilize = 10 * _minute
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    t2.settle_time = _hour * 1
    
    r2.add_test(t1)
    r2.add_test(t2)
    return runagpid(r2)

In [6]:
#r=run_80Ltests2()

In [7]:
# r2 = run_80Ltests3()

In [8]:
def run_80Ltests4():
    ip='71.189.82.196:85'
    p = 60
    i = 35
    d = 0
    app = hello3.open_hello(ip)
    app.retry_count = -1
    r2 = PIDRunner(wb_name='temp_80L_test_50L_a2a_60.35.0 worst case.xlsx')
    t1 = TempAutoToAutoTest(p, i, d, 37, 35.8, app)
    t1.pretest_stabilize = 10*_minute
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    t1.settle_time = _hour * 1
    
    t2 = TempAutoToAutoTest(p, i, d, 35.8, 37, app)   
    t2.pretest_stabilize = 10 * _minute
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    t2.settle_time = _hour * 1
    
    r2.add_test(t1)
    r2.add_test(t2)
    return runagpid(r2)

In [9]:
# f4 = run_80Ltests4()

In [10]:
def run_80Ltests5():
    ip='71.189.82.196:85'
    p = 60
    i = 19
    d = 0
    app = hello3.open_hello(ip)
    app.retry_count = -1
    r2 = PIDRunner(wb_name='temp_80L_test_50L_a2a_%d.%d.0 worst case.xlsx'%(p,i))
    t1 = TempAutoToAutoTest(p, i, d, 35.8, 37, app)
    t1.pretest_stabilize = 1 * _hour
    t1.pretest_stabilize_timeout = 3 * _hour
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    t1.settle_time = _hour * 1
    
    t2 = TempAutoToAutoTest(p, i, d, 35, 37, app)   
    t2.pretest_stabilize = 1 * _hour
    t2.pretest_stabilize_timeout = 3 * _hour
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    t2.settle_time = _hour * 1
    
    t3 = TempAutoToAutoTest(p, i, d, 37, 35.8, app)
    t3.pretest_stabilize = 10 * _minute
    t3.mintime = _hour * 2
    t3.error_band = 0.1
    t3.settle_time = _hour * 1
    
    t4 = TempAutoToAutoTest(p, i, d, 37, 35, app)
    t4.pretest_stabilize = 10 * _minute
    t4.mintime = _hour * 2
    t4.error_band = 0.1
    t4.settle_time = _hour * 1
    
    r2.add_test(t3)
    r2.add_test(t1)
    r2.add_test(t4)
    r2.add_test(t2)
    return runagpid(r2)

In [11]:
# r5 = run_80Ltests5()

In [14]:
def run_80Ltests7():
    ip='71.189.82.196:85'
    p = 60
    i = 22
    d = 0
    beta = 0
    app = hello3.open_hello(ip)
    app.login()
    app.setconfig("temperature", "beta", beta)
    app.retry_count = -1
    r2 = PIDRunner(wb_name='temp_80L_test_50L_all_%d.%d.0 beta %.1f.xlsx'%(p,i,beta))
    t1 = TempAutoToAutoTest(p, i, d, 35.8, 37, app)
    t1.pretest_stabilize = 1 * _hour
    t1.pretest_stabilize_timeout = 3 * _hour
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    t1.settle_time = _hour * 1
    
    t2 = TempAutoToAutoTest(p, i, d, 35, 37, app)   
    t2.pretest_stabilize = 1 * _hour
    t2.pretest_stabilize_timeout = 3 * _hour
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    t2.settle_time = _hour * 1
    
    t3 = TempAutoToAutoTest(p, i, d, 37, 35.8, app)
    t3.pretest_stabilize = 10 * _minute
    t3.mintime = _hour * 2
    t3.error_band = 0.1
    t3.settle_time = _hour * 1
    
    t4 = TempAutoToAutoTest(p, i, d, 37, 35, app)
    t4.pretest_stabilize = 10 * _minute
    t4.mintime = _hour * 2
    t4.error_band = 0.1
    t4.settle_time = _hour * 1
    
    t0 = TempOffToAutoTest(p, i, d, 26, 37, app)
    t0.pretest_stabilize_timeout   = 18 * _hour
    t0.pretest_stabilize_margin    = 0.2
    t0.settle_time                 = 60 * _minute
    t0.timeout                     = 3 * _hour
    t0.error_band                  = 0.1
    t0.mintime                     = 3 * _hour
    t0.wait_for_cooling            = True
    t0.agitation_sp                = 12
    
    for t in (t0, t1, t2, t3, t4):
        t.metadata['Beta'] = beta
    
    r2.add_test(t3)
    r2.add_test(t1)
    r2.add_test(t4)
    r2.add_test(t2)
    r2.add_test(t0)
    
    return runagpid(r2)

In [15]:
r7 = run_80Ltests7()

01/23/17 10:48:54 INFO <runall>: Running test 1 of 5: P:60.000 I:22.000 D:0.0000 SP:35
01/23/17 10:48:54 INFO <run>: Beginning test run for AutoToAuto Start:37.0 SP:35.8
01/23/17 10:48:54 INFO <setup_settings>: Setting PID Settings
01/23/17 10:48:56 INFO <setup_test>: Setting Temperature to AUTO 37 C and waiting until stable for 600 seconds (3600 second timeout)
01/23/17 10:48:56 INFO <auto>: Setting Temperature to 37C
01/23/17 10:48:58 INFO <wait_for_settle>: Waiting for PV to settle
01/23/17 10:48:58 INFO <wait_for_settle>: Update: PV: 36.96 SP: 37.00 Settle Remaining: 599 Timeout: 3599
01/23/17 10:49:59 INFO <wait_for_settle>: Update: PV: 36.93 SP: 37.00 Settle Remaining: 539 Timeout: 3539
01/23/17 10:50:59 INFO <wait_for_settle>: Update: PV: 36.91 SP: 37.00 Settle Remaining: 479 Timeout: 3479
01/23/17 10:51:59 INFO <wait_for_settle>: Update: PV: 36.90 SP: 37.00 Settle Remaining: 419 Timeout: 3419
01/23/17 10:52:59 INFO <wait_for_settle>: Update: PV: 36.89 SP: 37.00 Settle Remaining

ERROR OCCURRED DURING REQUEST
IPV4 ADDRESS: 71.189.82.196:85
REQUESTED URL: </webservice/interface/?&call=getMainValues&json=true>
MESSAGE: ConnectionError(('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None)))


01/23/17 12:26:10 INFO <wait_for_settle>: Update: PV: 35.79 SP: 35.80 Settle Remaining: 1183 Timeout: 5568
01/23/17 12:26:59 INFO <wait_for_settle>: Update: PV: 35.79 SP: 35.80 Settle Remaining: 1133 Timeout: 5519
01/23/17 12:27:59 INFO <wait_for_settle>: Update: PV: 35.80 SP: 35.80 Settle Remaining: 1074 Timeout: 5459
01/23/17 12:28:59 INFO <wait_for_settle>: Update: PV: 35.79 SP: 35.80 Settle Remaining: 1013 Timeout: 5399
01/23/17 12:29:59 INFO <wait_for_settle>: Update: PV: 35.79 SP: 35.80 Settle Remaining: 954 Timeout: 5339
01/23/17 12:30:59 INFO <wait_for_settle>: Update: PV: 35.80 SP: 35.80 Settle Remaining: 893 Timeout: 5279
01/23/17 12:31:59 INFO <wait_for_settle>: Update: PV: 35.80 SP: 35.80 Settle Remaining: 833 Timeout: 5219
01/23/17 12:32:59 INFO <wait_for_settle>: Update: PV: 35.80 SP: 35.80 Settle Remaining: 774 Timeout: 5159
01/23/17 12:33:59 INFO <wait_for_settle>: Update: PV: 35.80 SP: 35.80 Settle Remaining: 713 Timeout: 5099
01/23/17 12:34:59 INFO <wait_for_settle>: 

ERROR OCCURRED DURING REQUEST
IPV4 ADDRESS: 71.189.82.196:85
REQUESTED URL: </webservice/interface/?&call=getMainValues&json=true>
MESSAGE: ConnectionError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)))


01/23/17 12:56:05 INFO <wait_for_min_time>: Update: Waiting 173 seconds to finish test.
01/23/17 12:57:00 INFO <wait_for_min_time>: Update: Waiting 119 seconds to finish test.
01/23/17 12:58:00 INFO <wait_for_min_time>: Update: Waiting 59 seconds to finish test.
01/23/17 12:58:59 INFO <run>: Finished Test
01/23/17 12:58:59 INFO <runall>: Successful test
01/23/17 12:58:59 INFO <runall>: Running test 2 of 5: P:60.000 I:22.000 D:0.0000 SP:37
01/23/17 12:58:59 INFO <run>: Beginning test run for AutoToAuto Start:35.8 SP:37.0
01/23/17 12:58:59 INFO <setup_settings>: Setting PID Settings
01/23/17 12:59:03 INFO <setup_test>: Setting Temperature to AUTO 35 C and waiting until stable for 3600 seconds (10800 second timeout)
01/23/17 12:59:03 INFO <auto>: Setting Temperature to 35C
01/23/17 12:59:04 INFO <wait_for_settle>: Waiting for PV to settle
01/23/17 12:59:04 INFO <wait_for_settle>: Update: PV: 35.81 SP: 35.80 Settle Remaining: 3599 Timeout: 10799
01/23/17 13:00:04 INFO <wait_for_settle>: Up

In [25]:
def run_80Ltests8():
    ip='71.189.82.196:85'
    p = 60
    i = 20
    d = 0
    beta = 0
    app = hello3.open_hello(ip)
    app.login()
    app.set_mode('temperature', 1, 17)
    app.setconfig("temperature", "beta", beta)
    app.retry_count = -1
    r2 = PIDRunner(wb_name='temp_80L_test_50L_all_%d.%d.0 beta %.1f.xlsx'%(p,i,beta))
    t1 = TempAutoToAutoTest(p, i, d, 35.8, 37, app)
    t1.pretest_stabilize = 1 * _hour
    t1.pretest_stabilize_timeout = 3 * _hour
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    t1.settle_time = _hour * 1
    
    t2 = TempAutoToAutoTest(p, i, d, 35, 37, app)   
    t2.pretest_stabilize = 1 * _hour
    t2.pretest_stabilize_timeout = 3 * _hour
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    t2.settle_time = _hour * 1
    
    t3 = TempAutoToAutoTest(p, i, d, 37, 35.8, app)
    t3.pretest_stabilize = 10 * _minute
    t3.mintime = _hour * 2
    t3.error_band = 0.1
    t3.settle_time = _hour * 1
    
    t4 = TempAutoToAutoTest(p, i, d, 37, 35, app)
    t4.pretest_stabilize = 10 * _minute
    t4.mintime = _hour * 2
    t4.error_band = 0.1
    t4.settle_time = _hour * 1
    
    t0 = TempOffToAutoTest(p, i, d, 26, 37, app)
    t0.pretest_stabilize_timeout   = 18 * _hour
    t0.pretest_stabilize_margin    = 0.2
    t0.settle_time                 = 60 * _minute
    t0.timeout                     = 3 * _hour
    t0.error_band                  = 0.1
    t0.mintime                     = 3 * _hour
    t0.wait_for_cooling            = True
    t0.agitation_sp                = 12
    
    for t in (t0, t1, t2, t3, t4):
        t.metadata['Beta'] = beta
    
    r2.add_test(t3)
    r2.add_test(t1)
    r2.add_test(t4)
    r2.add_test(t2)
    r2.add_test(t0)
    
    return runagpid(r2)

In [ ]:
r8 = run_80Ltests8()

01/24/17 10:45:56 INFO <runall>: Running test 1 of 5: P:60.000 I:20.000 D:0.0000 SP:35
01/24/17 10:45:56 INFO <run>: Beginning test run for AutoToAuto Start:37.0 SP:35.8
01/24/17 10:45:56 INFO <setup_settings>: Setting PID Settings
01/24/17 10:45:57 INFO <setup_test>: Setting Temperature to AUTO 37 C and waiting until stable for 600 seconds (3600 second timeout)
01/24/17 10:45:57 INFO <auto>: Setting Temperature to 37C
01/24/17 10:45:58 INFO <wait_for_settle>: Waiting for PV to settle
01/24/17 10:45:58 INFO <wait_for_settle>: Update: PV: 36.99 SP: 37.00 Settle Remaining: 599 Timeout: 3599
